In [14]:
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib
import matplotlib.pyplot as plt
from torchvision.models import resnet34

In [15]:
#Checking for devoce
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [16]:
#Path for train, valid, test directory
train_path = 'content/train'
valid_path = 'content/valid'
test_path = 'content/test'

In [17]:
#Transforms
transformer = transforms.Compose([
    transforms.Resize((256, 256)),            # Resize image to 256x256
    transforms.RandomHorizontalFlip(),         # Random horizontal flip
    transforms.RandomVerticalFlip(),           # Random vertical flip (optional, use if needed)
    transforms.RandomRotation(10),             # Random rotation between -10 and 10 degrees
    transforms.ColorJitter(brightness=0.2,     # Randomly change brightness
                           contrast=0.2,       # Randomly change contrast
                           saturation=0.2,     # Randomly change saturation
                           hue=0.1),           # Randomly change hue
    transforms.ToTensor(),                     # Convert image to tensor
    transforms.Normalize(mean=[0.5, 0.5, 0.5],  # Normalize with mean and std
                         std=[0.5, 0.5, 0.5])
])

In [18]:
#DataLoader
train_loader = DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size=128, shuffle=True
)

valid_loader = DataLoader(
    torchvision.datasets.ImageFolder(valid_path,transform=transformer),
    batch_size=128, shuffle=True
)

In [19]:
#categories
root = pathlib.Path(train_path)
classes = sorted([j.name.split('/')[-1] for j in root.iterdir()])
print(classes)

['A', 'AA', 'AAA', 'B', 'Black', 'Borer Damaged', 'Elephant Ear', 'Fade', 'Half', 'Incomplete', 'Peaberry', 'Triangle']


In [20]:
model = resnet34(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 12)
model = model.to(device)

d:\Experiment\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Experiment\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [21]:
#Optimizer and loss function
optimizer = Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
loss_function = nn.CrossEntropyLoss()
num_epochs = 200

In [22]:
#Calculateing the size of trining and valid images
train_count = len(glob.glob(train_path+'/**/*.jpg'))
valid_count = len(glob.glob(valid_path+'/**/*.jpg'))

In [23]:
print(train_count,valid_count)

5760 720


In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torchvision import datasets, transforms
from torchvision.models import resnet34

# Initialization
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
best_accuracy = 0.0

# Load your dataset
# train_loader and test_loader should be defined before

# Define the model
model = resnet34(num_classes=12)  # Replace 12 with the number of your classes
model.fc = nn.Linear(model.fc.in_features, 12)  # Replace 12 with the number of your classes
model.to(device)

# Define loss function and optimizer
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

train_count = len(train_loader.dataset)
valid_count = len(valid_loader.dataset)

for epoch in range(num_epochs):

    # Training phase
    model.train()
    train_accuracy = 0.0
    train_loss = 0.0

    for i, (images, labels) in enumerate(valid_loader):
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.size(0)
        _, prediction = torch.max(outputs.data, 1)
        train_accuracy += int(torch.sum(prediction == labels.data))

    train_accuracy = train_accuracy / train_count
    train_loss = train_loss / train_count

    # Evaluation phase
    model.eval()
    valid_accuracy = 0.0
    valid_loss = 0.0

    with torch.no_grad():
        for i, (images, labels) in enumerate(valid_loader):
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = loss_function(outputs, labels)
            valid_loss += loss.item() * images.size(0)
            _, prediction = torch.max(outputs.data, 1)
            valid_accuracy += int(torch.sum(prediction == labels.data))

    valid_accuracy = valid_accuracy / valid_count
    valid_loss = valid_loss / valid_count

    print(f'Epoch: {epoch} Train Loss: {train_loss:.4f} Train Accuracy: {train_accuracy:.4f} Valid Loss: {valid_loss:.4f} Valid Accuracy: {valid_accuracy:.4f}')

    # Save the best model
    if valid_accuracy > best_accuracy:
        torch.save(model.state_dict(), 'content/resnet34')
        best_accuracy = valid_accuracy
        print("--------------")
        print(best_accuracy)

Epoch: 0 Train Loss: 0.3488 Train Accuracy: 0.0109 Valid Loss: 30.9234 Valid Accuracy: 0.0750
--------------
0.075
Epoch: 1 Train Loss: 0.3085 Train Accuracy: 0.0130 Valid Loss: 5.5081 Valid Accuracy: 0.0889
--------------
0.08888888888888889
Epoch: 2 Train Loss: 0.2938 Train Accuracy: 0.0200 Valid Loss: 3.6822 Valid Accuracy: 0.1389
--------------
0.1388888888888889
Epoch: 3 Train Loss: 0.2802 Train Accuracy: 0.0215 Valid Loss: 3.5833 Valid Accuracy: 0.1458
--------------
0.14583333333333334
Epoch: 4 Train Loss: 0.2730 Train Accuracy: 0.0260 Valid Loss: 2.7998 Valid Accuracy: 0.1806
--------------
0.18055555555555555
Epoch: 5 Train Loss: 0.2469 Train Accuracy: 0.0293 Valid Loss: 8.2477 Valid Accuracy: 0.0514
Epoch: 6 Train Loss: 0.2375 Train Accuracy: 0.0337 Valid Loss: 6.3069 Valid Accuracy: 0.1042
Epoch: 7 Train Loss: 0.2315 Train Accuracy: 0.0345 Valid Loss: 2.5347 Valid Accuracy: 0.2125
--------------
0.2125
Epoch: 8 Train Loss: 0.2294 Train Accuracy: 0.0408 Valid Loss: 2.4744 Val

In [25]:
PATH = 'content/resnet34.pth'
print("\nSaving the model...")
torch.save(model, PATH)
print('Saved!')


Saving the model...
Saved!


In [26]:
torch.cuda.empty_cache()  # ล้างหน่วยความจำ CUDA